# Prepare Env

- 2 Linux machines with `patroni`, `postgres`, `pgBackRest`, `ssh-server` installed -> pg-1, pg-2
- 2 Linux machines with `etcd` installed and a cluster setup -> etcd-1, etcd-2
- 1 Linux machine with `pgBackRest` and `ssh-server` installed -> bcp-repo

# Idea

- Mix HA & LB with backup strategies
- Use pgBackRest as a powerful replica creation tool

# Demo

Since there are 2 or more machines that construct the postgres cluster, the repository should be accessible from all the machines, so we create a dedicated remote repository machine.

## Setup Passwordless SSH

### Start SSH Server

In [ ]:
### On Postgres And Repo ###
service ssh start

### Setup Password For SSH

In [ ]:
### On Postgres And Repo ###
PG_USER_PASS=123
echo "postgres:$PG_USER_PASS" | chpasswd

### Create Keys

In [ ]:
### On Postgres And Repo ###
su postgres

mkdir -m 750 -p /var/lib/postgresql/.ssh
ssh-keygen -f /var/lib/postgresql/.ssh/id_rsa -t rsa -b 4096 -N ""

### Exchange Keys

In [ ]:
### On bcp-repo ###
# For each postgres node
PG_USER_PASS=123
(echo -n 'no-agent-forwarding,no-X11-forwarding,no-port-forwarding,' && \
    echo -n 'command="/usr/bin/pgbackrest ${SSH_ORIGINAL_COMMAND#* }" ' && \
    sshpass -p $PG_USER_PASS ssh -o StrictHostKeyChecking=accept-new postgres@<pg-host> cat /var/lib/postgresql/.ssh/id_rsa.pub) | \
    tee -a /var/lib/postgresql/.ssh/authorized_keys

### On pg nodes ###
PG_USER_PASS=123
(echo -n 'no-agent-forwarding,no-X11-forwarding,no-port-forwarding,' && \
       echo -n 'command="/usr/bin/pgbackrest ${SSH_ORIGINAL_COMMAND#* }" ' && \
       sshpass -p $PG_USER_PASS ssh -o StrictHzostKeyChecking=accept-new postgres@bcp-repo cat /var/lib/postgresql/.ssh/id_rsa.pub) | \
       tee -a /var/lib/postgresql/.ssh/authorized_keys

## Configure pgBackRest Repo

In [ ]:
### On bcp-repo ###
cat << EOF > /etc/pgbackrest.conf
[global]
repo1-path=/var/lib/pgbackrest
repo1-retention-full=2
repo1-host-user=postgres
start-fast=y
delta=y

[main]
pg1-host=pg-1
pg1-path=/var/lib/postgresql/16/main
pg1-port=5432
EOF

## Configure pgBackRest on Postgres Nodes 

In [ ]:
#### On Postgres Nodes ####
cat << EOF > /etc/pgbackrest.conf
[demo]
pg1-path=/var/lib/postgresql/16/main

[global]
log-level-file=detail
repo1-host=bcp-repo
EOF

## Create Backup

In [ ]:
#### On repo-bcp ####
su postgres -c "pgbackrest --stanza=main stanza-create"

TODO - https://pgbackrest.org/user-guide.html#replication\
https://patroni.readthedocs.io/en/latest/replica_bootstrap.html

### Setting Up WAL Archiving For Backup

To set up 

As you can see there is a simple archiving mechanism in the defaults using `cp` command for archiving and restoring.

Try using `pgBackRest` as your archiving and backup tool as learnt in `Backup` lesson

*Hint* - You n

```yaml
postgresql:
    create_replica_methods:
        - pgbackrest
        - basebackup
    pgbackrest:
        command: /usr/bin/pgbackrest --stanza=<scope> --delta restore
        keep_data: True
        no_params: True
    basebackup:
        max-rate: '100M'
```